In [38]:
!unzip CEDAR.zip

unzip:  cannot find or open CEDAR.zip, CEDAR.zip.zip or CEDAR.zip.ZIP.


In [39]:
import glob

#read one folder
files = glob.glob('CEDAR/1/*.png')

In [40]:
files

['CEDAR/1/forgeries_1_1.png',
 'CEDAR/1/forgeries_1_10.png',
 'CEDAR/1/forgeries_1_11.png',
 'CEDAR/1/forgeries_1_12.png',
 'CEDAR/1/forgeries_1_13.png',
 'CEDAR/1/forgeries_1_14.png',
 'CEDAR/1/forgeries_1_15.png',
 'CEDAR/1/forgeries_1_16.png',
 'CEDAR/1/forgeries_1_17.png',
 'CEDAR/1/forgeries_1_18.png',
 'CEDAR/1/forgeries_1_19.png',
 'CEDAR/1/forgeries_1_2.png',
 'CEDAR/1/forgeries_1_20.png',
 'CEDAR/1/forgeries_1_21.png',
 'CEDAR/1/forgeries_1_22.png',
 'CEDAR/1/forgeries_1_23.png',
 'CEDAR/1/forgeries_1_24.png',
 'CEDAR/1/forgeries_1_3.png',
 'CEDAR/1/forgeries_1_4.png',
 'CEDAR/1/forgeries_1_5.png',
 'CEDAR/1/forgeries_1_6.png',
 'CEDAR/1/forgeries_1_7.png',
 'CEDAR/1/forgeries_1_8.png',
 'CEDAR/1/forgeries_1_9.png',
 'CEDAR/1/original_1_1.png',
 'CEDAR/1/original_1_10.png',
 'CEDAR/1/original_1_11.png',
 'CEDAR/1/original_1_12.png',
 'CEDAR/1/original_1_13.png',
 'CEDAR/1/original_1_14.png',
 'CEDAR/1/original_1_15.png',
 'CEDAR/1/original_1_16.png',
 'CEDAR/1/original_1_17.pn

In [41]:
from itertools import combinations

combinations_list = list(combinations(files, 2))

# Print each combination
print(combinations_list)

[('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_10.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_11.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_12.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_13.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_14.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_15.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_16.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_17.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_18.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_19.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_2.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_20.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_21.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_22.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_23.png'), ('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_24.png'), ('CEDAR/1/forgeries_1_1.

In [42]:
#one value of combination
x,y=combinations_list[0]

In [43]:
x.split("/")[-1][0] =="o"

False

In [44]:
label1=0 if "f" ==  x.split("/")[-1][0] else 1
label2=0 if "f" ==  y.split("/")[-1][0] else 1
label=label1*label2

In [45]:
label1

0

In [46]:
x,y

('CEDAR/1/forgeries_1_1.png', 'CEDAR/1/forgeries_1_10.png')

In [47]:
label

0

# general function

In [48]:
import pandas as pd
import glob
from itertools import combinations

def cedar_files_to_csv(n, file_type):
    image1 = []
    image2 = []
    label = []
    labels1=[]
    labels2=[]
    
    for i in range(1, n+1):
        files = glob.glob(f'CEDAR/{i}/*.png')
        pairs = list(combinations(files, 2))
        
        for combo in pairs:
            # print(combo[0].split("\\")[-1][0])
            label1 = 0 if "f" ==  combo[0].split("/")[-1][0]  else 1
            label2 = 0 if "f" == combo[1].split("/")[-1][0]  else 1
            labels1.append(label1)
            labels2.append(label2)
            label.append(label1 * label2)
            image1.append(combo[0])
            image2.append(combo[1])
    
    # Create the DataFrame
    df = pd.DataFrame(list(zip(image1, image2, labels1,labels2,label)),  columns=['Testing_Signature', 'Original_Signature','Label1','Label2', 'Ouput'])
    print(df.shape)
    print(df.Label2.value_counts())
    df=df[df.Label2!=0]
    print(df.shape)
    print(df.Label2.value_counts())
    df.to_csv("data/CEDAR_extracted.csv",index=False)

In [49]:
cedar_files_to_csv(55,"png")

(62040, 5)
Label2
1    46860
0    15180
Name: count, dtype: int64
(46860, 5)
Label2
1    46860
Name: count, dtype: int64


In [50]:
df1=pd.read_csv("data/CEDAR_extracted.csv")

In [51]:
df1.shape

(46860, 5)

In [52]:
df1.shape

(46860, 5)

In [53]:
df1.head()

,Testing_Signature,Original_Signature,Label1,Label2,Ouput
0,CEDAR/1/forgeries_1_1.png,CEDAR/1/original_1_1.png,0,1,0
1,CEDAR/1/forgeries_1_1.png,CEDAR/1/original_1_10.png,0,1,0
2,CEDAR/1/forgeries_1_1.png,CEDAR/1/original_1_11.png,0,1,0
3,CEDAR/1/forgeries_1_1.png,CEDAR/1/original_1_12.png,0,1,0
4,CEDAR/1/forgeries_1_1.png,CEDAR/1/original_1_13.png,0,1,0


In [54]:
from sklearn.model_selection import train_test_split
#split the df in train, val, test in 80:10:10 and save it as csv files
X_train, X_test, y_train, y_test = train_test_split(df1[['Testing_Signature', 'Original_Signature']], df1['Ouput'], test_size=0.2, random_state=42,
                                                    shuffle=True, stratify=df1['Ouput'])
X_train, X_val, y_train, y_val = train_test_split(df1[['Testing_Signature', 'Original_Signature']], df1['Ouput'], test_size=0.25, random_state=42,
                                                    shuffle=True, stratify=df1['Ouput'])  #0.25*0.8=0.2
train=pd.DataFrame(list(zip(X_train['Testing_Signature'],X_train['Original_Signature'],y_train)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])
val=pd.DataFrame(list(zip(X_val['Testing_Signature'],X_val['Original_Signature'],y_val)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])
test=pd.DataFrame(list(zip(X_test['Testing_Signature'],X_test['Original_Signature'],y_test)),  columns=['Testing_Signature', 'Original_Signature', 'Ouput'])



In [55]:
train['Ouput'].value_counts()

Ouput
0    23760
1    11385
Name: count, dtype: int64

In [56]:
val['Ouput'].value_counts()

Ouput
0    7920
1    3795
Name: count, dtype: int64

In [57]:
test['Ouput'].value_counts()

Ouput
0    6336
1    3036
Name: count, dtype: int64

In [58]:
train.to_csv("data/CEDAR_train.csv",index=False)
val.to_csv("data/CEDAR_val.csv",index=False)
test.to_csv("data/CEDAR_test.csv",index=False)